In [2]:
from pathlib import Path
import concurrent.futures
from PIL import Image
from tqdm import tqdm

In [3]:
DATA_DIR = Path("/workspaces/cv/data/surveys/roc2")
THUMBSIZE = (600, 400)  # 9504x6336
THUMBQUALITY = 80

In [10]:
# tasks = []
# for survey_dir in sorted(DATA_DIR.iterdir()):
#     for flight_dir in sorted(survey_dir.iterdir()):
#         if not flight_dir.is_dir():
#             continue
#         assert flight_dir.name.startswith("flight")
#         photodir = flight_dir / "photos"
#         if not photodir.exists():
#             continue
#         thumdir = flight_dir / "thumbnails"
#         for imgpath in sorted(photodir.iterdir()):
#             assert imgpath.name.endswith(".JPG")
#             tasks.append((imgpath, thumdir / f"{imgpath.stem}.jpg"))
# # tasks = tasks[:10]
# print(len(tasks))
tasks = []
for survey_dir in sorted(DATA_DIR.iterdir()):
    for flight_dir in sorted(survey_dir.iterdir()):
        if not flight_dir.is_dir():
            continue
        assert flight_dir.name.startswith("flight")
        for photodir in sorted((flight_dir / "DCIM").iterdir()):
            thumdir = photodir.parent / f"{photodir.name}.thumbnails"
            for imgpath in sorted(photodir.iterdir()):
                assert imgpath.name.endswith(".JPG")
                tasks.append((imgpath, thumdir / f"{imgpath.stem}.jpg"))
# tasks = tasks[:10]
print(len(tasks))

9718


In [12]:
def do(imgpath, thumbpath):
    img = Image.open(imgpath)
    img.thumbnail(THUMBSIZE)
    thumbpath.parent.mkdir(parents=True, exist_ok=True)
    img.save(thumbpath, quality=THUMBQUALITY)


with concurrent.futures.ThreadPoolExecutor(max_workers=128) as ex:
    futures = [ex.submit(do, *args) for args in tasks]
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        res = future.result()

100%|██████████| 9718/9718 [09:49<00:00, 16.49it/s] 
